Чисто небольшой пример, как работать с датасетами из кортинок, оче примитивно!

Датасет на [каггле](https://www.kaggle.com/datasets/muratkokludataset/rice-image-dataset), распакуйте в папку с семинаром.

In [ ]:
from torchvision import datasets, transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
%matplotlib inline

В трансформации я даже нормализации не положила, минимум миниморум - в тензора загнать

In [ ]:
transform = transforms.Compose(
    [
        transforms.ToTensor()
    ]
)

dataset = datasets.ImageFolder('Rice_Image_Dataset', transform=transform) # указываете путь к своему датасету и вуаля

Очень примитивный способ собрать датасеты без классов даже

In [ ]:
def train_val_dataset(dataset, val_split=0.25):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    datasets = {}
    datasets['train'] = Subset(dataset, train_idx)
    datasets['val'] = Subset(dataset, val_idx)
    return datasets

In [ ]:
traintest = train_val_dataset(dataset)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(traintest['train'], batch_size=64, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(traintest['val'], batch_size=64, shuffle=False)

Позырить че собралось - картинка в матрице + ее класс (как бы номер папки, в которой она лежала)

In [ ]:
traintest['train'][2]

In [ ]:
def train(model, optimizer, n_epochs=5):
    for epoch in range(n_epochs):
        # тренировка
        for x_train, y_train in tqdm(train_dataloader):
            y_pred = model(x_train)
            loss = F.cross_entropy(y_pred, y_train)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        # валидация
        if epoch % 2 == 0:
            val_loss = []
            val_accuracy = []
            with torch.no_grad():
                for x_val, y_val in tqdm(val_dataloader): # tqdm фигачит лепоту с ползунками
                    y_pred = model(x_val)
                    loss = F.cross_entropy(y_pred, y_val)
                    val_loss.append(loss.numpy())
                    val_accuracy.extend((torch.argmax(y_pred, dim=-1) == y_val).numpy().tolist())

            # печатаем метрики
            print(f"Epoch: {epoch}, loss: {np.mean(val_loss)}, accuracy: {np.mean(val_accuracy)}")

Про вычисления. 

- Картинки у нас 250х250 размером, в три канала (хотя ирл они там чб). 
- Размер ядра 5, следовательно, станет 250 - 4 = 246х246
- Уменьшим в 4 раза = 61х61
- Не забываем, что все это время у нас еще 10 ядер
- Когда разворачиваем, выходит колбасень длиной в 61х61х10 = 37 210
- дальше уже фигачим FC

In [ ]:
model = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=10, kernel_size=5), # добавим сверточный слой с 10 ядрами
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=4),  # уменьшим картинку в 4 раза по каждой стороне
    nn.Flatten(),                 # превращаем картинку 61х61х10 в вектор размером 37 210
    nn.Linear(61*61*10, 128),       # линейный слой, преобразующий вектор размера 37 210 в вектор размера 128
    nn.ReLU(),                    # нелинейность
    nn.Linear(128, 5),           # отображение в пространство рамерности числа меток классификации
)

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [ ]:
train(model, optimizer)